In [1]:
# Dependency conflicts
!pip install dask==2021.4.0 sparse==0.9.1

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 941.1/941.1 kB 6.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: sparse
    Found existing installation: sparse 0.15.1
    Uninstalling sparse-0.15.1:
      Successfully uninstalled sparse-0.15.1
  Attempting uninstall: dask
    Found existing installation: dask 2023.11.0
    Uninstalling dask-2023.11.0:
      Successfully uninstalled dask-2023.11.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2023.11.0 requires dask==2023.11.0, but you have dask 2021.4.0 which is incompatible.
rapids-dask-dependency 23.12.1 requires dask==2023.11.0, but you have dask 2021.4.0 which is incompatible.


In [1]:
import os
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
from scipy.sparse import csr_matrix, issparse
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import pandas as pd
import numpy as np
import random
import sc_toolbox
import decoupler as dc
import rpy2.robjects as ro
import diffxpy.api as de
import batchglm
import rpy2.rinterface_lib.callbacks
import anndata2ri
import logging

from rpy2.robjects import pandas2ri
from rpy2.robjects import r

from batchglm.pkg_constants import TF_CONFIG_PROTO

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

sc.settings.verbosity = 0
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython
os.chdir('/root/host_home')

2024-06-28 10:46:00.396598: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
%%R
library(edgeR)
library(scater)
library(MAST)
library(Seurat)
library(DESeq2)
library(data.table)
library(dplyr)
#library(EnhancedVolcano)
library(apeglm)
library(lme4)


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

Loading required package: limma
Loading required package: SingleCellExperiment
Loading required package: SummarizedExperiment
Loading required package: MatrixGenerics
Loading required package: matrixStats

Attaching package: ‘MatrixGenerics’

The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2

In [3]:
plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
#sc.logging.print_versions()



#logging.getLogger("tensorflow").setLevel(logging.ERROR)
#logging.getLogger("batchglm").setLevel(logging.INFO)


pd.set_option('display.max_rows', 500)
#warnings.filterwarnings("ignore", category=DeprecationWarning, module="tensorflow")

In [4]:
#Set number of threads
TF_CONFIG_PROTO.inter_op_parallelism_threads = 1
TF_CONFIG_PROTO.intra_op_parallelism_threads = 12

# Filenames

In [5]:
#input
adata_input = 'NPCs_merge/processed_data/NPC_clustered_scvi_nb_cuda.h5ad'

In [6]:
#output
save_dir = 'NPCs_merge/tables/' 
resolution = 'cell_types'

# Read in data

In [7]:
adata = sc.read(adata_input)


In [8]:
adata

AnnData object with n_obs × n_vars = 61400 × 17366
    obs: 'Donor', 'lane', 'condition', 'age', 'years_of_illness', 'deltaHY', 'deltaUPDRS', 'deltaADL', 'deltaDOPA', 'gender', 'cultivation_period', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'n_counts', 'S_score', 'G2M_score', 'phase', 'cell_types', 'cell_types_abbrev', 'size_factors', 'donor_batch_ID', '_scvi_batch', '_scvi_labels', 'Ferroptosis_pos_corr', 'Ferroptosis_neg_corr', 'Apoptosis_pos_corr', 'Apoptosis_neg_corr', 'Necroptosis_pos_corr', 'Autophagy_pos_corr', 'Pathanatos_pos_corr'
    var: 'gene_ids', 'feature_types', 'genome', 'gene_symbols', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_to

# Preprocess data

In [9]:
# Clean up cell type names
adata.obs['cell_types_used'] = [ct.replace(" ", "-") for ct in adata.obs[resolution]]
adata.obs['cell_types_used'] = [ct.replace("+", "") for ct in adata.obs['cell_types_used']]

In [10]:
adata.obs["Donor"] = adata.obs["Donor"].astype("category")
adata.obs["condition"] = adata.obs["condition"].astype("category")
adata.obs['cell_types_used'] = adata.obs['cell_types_used'].astype("category")

In [11]:
adata.obs['cell_types_used'].cat.categories

Index(['Apop.-NCSC', 'Apop.-NSC', 'Glial-precursors', 'Immature-neurons',
       'NCSC', 'NSC1a', 'NSC1b', 'NSC2a', 'NSC2b'],
      dtype='object')

# Check the data

In [11]:
np.mean(adata.layers['soupX_counts'].astype(int) != adata.layers['soupX_counts'])

0.0

Counts are actually all integer counts.

In [12]:
adata.obs.condition.value_counts()

condition
Parkinson    38032
Control      23368
Name: count, dtype: int64

In [13]:
adata.obs.cultivation_period.value_counts()

cultivation_period
lp    30843
hp    30557
Name: count, dtype: int64

In [14]:
adata.obs.groupby(['cell_types_used'])['cultivation_period'].value_counts()

cell_types_used   cultivation_period
Apop.-NCSC        hp                      211
                  lp                      115
Apop.-NSC         lp                     2481
                  hp                     1385
Glial-precursors  hp                      991
                  lp                      896
Immature-neurons  lp                      910
                  hp                      504
NCSC              hp                     5004
                  lp                     2860
NSC1a             lp                    13980
                  hp                     8693
NSC1b             lp                     6940
                  hp                     6199
NSC2a             hp                     5647
                  lp                     2228
NSC2b             hp                     1923
                  lp                      433
Name: count, dtype: int64

In [15]:
adata.obs.groupby(['cell_types_used'])['condition'].value_counts()

cell_types_used   condition
Apop.-NCSC        Control        167
                  Parkinson      159
Apop.-NSC         Parkinson     2162
                  Control       1704
Glial-precursors  Parkinson     1088
                  Control        799
Immature-neurons  Parkinson      881
                  Control        533
NCSC              Control       4416
                  Parkinson     3448
NSC1a             Parkinson    14007
                  Control       8666
NSC1b             Parkinson     8730
                  Control       4409
NSC2a             Parkinson     5691
                  Control       2184
NSC2b             Parkinson     1866
                  Control        490
Name: count, dtype: int64

In [16]:
adata.obs.groupby(['cell_types_used'])['Donor'].value_counts()

cell_types_used   Donor          
Apop.-NCSC        UKERi88H-R1-002      68
                  UKERi1E4-R1-003      65
                  UKERi82A-R1-002      63
                  UKERiO3H-R1-005      32
                  UKERiAY6-R1-003      29
                  UKERiC99-R1-007      20
                  UKERiPX7-R1-001      13
                  UKERiJ2C-R1-015      11
                  UKERiM89-R1-005       9
                  UKERiR66-R1-007       9
                  UKERiG3G-R1-039       4
                  UKERi1JF-R1-018       3
Apop.-NSC         UKERiO3H-R1-005     465
                  UKERiPX7-R1-001     461
                  UKERiJ2C-R1-015     426
                  UKERiG3G-R1-039     424
                  UKERi82A-R1-002     384
                  UKERiAY6-R1-003     380
                  UKERi1E4-R1-003     262
                  UKERiR66-R1-007     249
                  UKERiC99-R1-007     244
                  UKERi88H-R1-002     210
                  UKERiM89-R1-005     192


In [17]:
adata.obs['condition_period'] = adata.obs['condition'].astype(str)+'_'+adata.obs['cultivation_period'].astype(str)

In [18]:
adata.obs['donor_period'] = adata.obs['Donor'].astype(str)+'_'+adata.obs['cultivation_period'].astype(str)

Note:
- General imbalance between Parkinson and control cell numbers


# Single-cell specific using diffxpy

In [19]:
if not os.path.exists(save_dir+'diffxpy'):
    # Create a new directory because it does not exist
    os.mkdir(save_dir+'diffxpy')
    print("The new directory: "+save_dir+"diffxpy is created!")
else: 
    print("The directory already exists")

The directory already exists


## Differential expression testing (overall)

### Cell type specific DE testing

In [20]:
de_results_overall = dict()


In [21]:
de.utils.preview_coef_names(
    sample_description=adata.obs,
    formula="~1 + cultivation_period + donor_period", 
)

['Intercept',
 'cultivation_period[T.hp]',
 'donor_period[T.UKERi1E4-R1-003_lp]',
 'donor_period[T.UKERi1JF-R1-018_hp]',
 'donor_period[T.UKERi1JF-R1-018_lp]',
 'donor_period[T.UKERi82A-R1-002_hp]',
 'donor_period[T.UKERi82A-R1-002_lp]',
 'donor_period[T.UKERi88H-R1-002_hp]',
 'donor_period[T.UKERi88H-R1-002_lp]',
 'donor_period[T.UKERiAY6-R1-003_hp]',
 'donor_period[T.UKERiAY6-R1-003_lp]',
 'donor_period[T.UKERiC99-R1-007_hp]',
 'donor_period[T.UKERiC99-R1-007_lp]',
 'donor_period[T.UKERiG3G-R1-039_hp]',
 'donor_period[T.UKERiG3G-R1-039_lp]',
 'donor_period[T.UKERiJ2C-R1-015_hp]',
 'donor_period[T.UKERiJ2C-R1-015_lp]',
 'donor_period[T.UKERiM89-R1-005_hp]',
 'donor_period[T.UKERiM89-R1-005_lp]',
 'donor_period[T.UKERiO3H-R1-005_hp]',
 'donor_period[T.UKERiO3H-R1-005_lp]',
 'donor_period[T.UKERiPX7-R1-001_hp]',
 'donor_period[T.UKERiPX7-R1-001_lp]',
 'donor_period[T.UKERiR66-R1-007_hp]',
 'donor_period[T.UKERiR66-R1-007_lp]']

In [22]:
for clust in adata.obs['cell_types_used'].cat.categories:
    adata_tmp = adata[adata.obs['cell_types_used'] == clust,:].copy()

    print(f'In cluster {clust}:')
    print(adata_tmp.obs.condition.value_counts())

    # Filter out genes to reduce multiple testing burden
    sc.pp.filter_genes(adata_tmp, min_cells=50)
    print(f'Testing {adata_tmp.n_vars} genes...')
    print("")
    
    test_tmp = de.test.wald(
    data=adata_tmp.layers['soupX_counts'],
    formula_loc="~ 1 + cultivation_period + donor_period + size_factors",
    as_numeric=['size_factors'],
    coef_to_test=['cultivation_period[T.hp]'],
    sample_description=adata_tmp.obs,
    gene_names=adata_tmp.var['gene_ids'].astype(str),
    constraints_loc={'donor_period':'cultivation_period'},
    noise_model='nb',
    dtype="float64"
    )
    
    #Store the results
    de_results_overall[clust] = test_tmp
    

In cluster Apop.-NCSC:
condition
Control      167
Parkinson    159
Name: count, dtype: int64
filtered out 13679 genes that are detected in less than 50 cells


Built constraints: donor_period[UKERi1E4-R1-003_lp]+donor_period[UKERi1JF-R1-018_lp]+donor_period[UKERi82A-R1-002_lp]+donor_period[UKERi88H-R1-002_lp]+donor_period[UKERiAY6-R1-003_lp]+donor_period[UKERiC99-R1-007_lp]+donor_period[UKERiG3G-R1-039_lp]+donor_period[UKERiJ2C-R1-015_lp]+donor_period[UKERiM89-R1-005_lp]+donor_period[UKERiO3H-R1-005_lp]+donor_period[UKERiPX7-R1-001_lp]+donor_period[UKERiR66-R1-007_lp]=0, donor_period[UKERi1E4-R1-003_hp]+donor_period[UKERi82A-R1-002_hp]+donor_period[UKERi88H-R1-002_hp]+donor_period[UKERiAY6-R1-003_hp]+donor_period[UKERiC99-R1-007_hp]+donor_period[UKERiJ2C-R1-015_hp]+donor_period[UKERiO3H-R1-005_hp]+donor_period[UKERiPX7-R1-001_hp]+donor_period[UKERiR66-R1-007_hp]=0


Testing 3687 genes...

training location model: True
training scale model: True
iter   0: ll=4398310.264567
iter   1: ll=4327930.710587, converged: 0.00% (loc: 4.26%, scale update: False), in 1.02sec
iter   2: ll=4298650.649053, converged: 0.00% (loc: 7.51%, scale update: False), in 1.11sec
iter   3: ll=4291106.512162, converged: 0.00% (loc: 7.51%, scale update: False), in 1.57sec
iter   4: ll=4288966.957398, converged: 0.00% (loc: 7.51%, scale update: False), in 1.32sec
iter   5: ll=4288312.843824, converged: 0.00% (loc: 7.51%, scale update: False), in 1.18sec
Fitting 3687 dispersion models: (progress not available with multiprocessing)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


iter   6: ll=1200551.432089, converged: 0.00% (loc: 0.00%, scale update: True), in 8.04sec
iter   7: ll=1198339.612272, converged: 0.00% (loc: 6.75%, scale update: False), in 1.36sec
iter   8: ll=1197695.801164, converged: 0.00% (loc: 8.76%, scale update: False), in 1.27sec
iter   9: ll=1197497.892091, converged: 0.00% (loc: 10.90%, scale update: False), in 1.31sec
iter  10: ll=1197445.177564, converged: 0.00% (loc: 18.14%, scale update: False), in 1.26sec
iter  11: ll=1197421.502611, converged: 0.00% (loc: 24.27%, scale update: False), in 1.22sec
Fitting 3687 dispersion models: (progress not available with multiprocessing)

...

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


iter  48: ll=26990700.995267, converged: 99.95% (loc: 99.95%, scale update: True), in 9.78sec
caught 2 linalg singular matrix errors
iter  49: ll=26990700.995242, converged: 99.95% (loc: 99.99%, scale update: False), in 0.60sec
iter  50: ll=26990700.995238, converged: 99.95% (loc: 99.99%, scale update: False), in 0.42sec
iter  51: ll=26990700.995238, converged: 99.95% (loc: 99.99%, scale update: False), in 0.42sec
iter  52: ll=26990700.995238, converged: 99.95% (loc: 99.99%, scale update: False), in 0.42sec
iter  53: ll=26990700.995238, converged: 99.95% (loc: 100.00%, scale update: False), in 0.45sec
Fitting 7 dispersion models: (progress not available with multiprocessing)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


iter  54: ll=26990700.995238, converged: 99.99% (loc: 99.99%, scale update: True), in 7.53sec
iter  55: ll=26990700.995238, converged: 99.99% (loc: 100.00%, scale update: False), in 0.42sec
iter  56: ll=26990700.995238, converged: 100.00% (loc: 100.00%, scale update: True), in 0.39sec


/usr/local/lib/python3.10/dist-packages/dask/array/core.py:2894: RuntimeWarning: divide by zero encountered in divide
  size = (limit / dtype.itemsize / largest_block) ** (1 / len(autos))


### Bulk-like DE testing across all clusters

In [ ]:
# Make a copy for the testing
adata_test = adata.copy()

print(adata_test.obs.condition.value_counts())

# Filter out genes to reduce multiple testing burden
sc.pp.filter_genes(adata_test, min_cells=50)
print(f'Testing {adata_test.n_vars} genes...')
print("")
    
test_bulk_overall = de.test.wald(
    data=adata_test.layers['soupX_counts'],
    formula_loc="~ 1 + cultivation_period + donor_period + size_factors",
    as_numeric=['size_factors'],
    coef_to_test=['cultivation_period[T.hp]'],
    sample_description=adata_test.obs,
    gene_names=adata_test.var['gene_ids'].astype(str),
    constraints_loc={'donor_period':'cultivation_period'},
    noise_model='nb',
    dtype="float64"
)


condition
Parkinson    38032
Control      23368
Name: count, dtype: int64
filtered out 670 genes that are detected in less than 50 cells
Testing 16696 genes...



Built constraints: donor_period[UKERi1E4-R1-003_lp]+donor_period[UKERi1JF-R1-018_lp]+donor_period[UKERi82A-R1-002_lp]+donor_period[UKERi88H-R1-002_lp]+donor_period[UKERiAY6-R1-003_lp]+donor_period[UKERiC99-R1-007_lp]+donor_period[UKERiG3G-R1-039_lp]+donor_period[UKERiJ2C-R1-015_lp]+donor_period[UKERiM89-R1-005_lp]+donor_period[UKERiO3H-R1-005_lp]+donor_period[UKERiPX7-R1-001_lp]+donor_period[UKERiR66-R1-007_lp]=0, donor_period[UKERi1E4-R1-003_hp]+donor_period[UKERi1JF-R1-018_hp]+donor_period[UKERi82A-R1-002_hp]+donor_period[UKERi88H-R1-002_hp]+donor_period[UKERiAY6-R1-003_hp]+donor_period[UKERiC99-R1-007_hp]+donor_period[UKERiG3G-R1-039_hp]+donor_period[UKERiJ2C-R1-015_hp]+donor_period[UKERiM89-R1-005_hp]+donor_period[UKERiO3H-R1-005_hp]+donor_period[UKERiPX7-R1-001_hp]+donor_period[UKERiR66-R1-007_hp]=0
/usr/local/lib/python3.10/dist-packages/batchglm/models/base_glm/utils.py:158: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warnin

training location model: True
training scale model: True
iter   0: ll=775942732.966850


### Post-process data

In [23]:
# extract gene symbol - gene_ids informations for later conversion
df_var = pd.DataFrame(adata.var)
df_var = df_var[['gene_symbols', 'gene_ids']]
df_var.reset_index(inplace=True) 
df_var = df_var.drop(columns='index')

In [24]:
de_signif_overall = dict()
de_full_res_overall = dict()

for clust in de_results_overall:
    tmp = de_results_overall[clust].summary()
    
    # add gene symbols
    tmp = tmp.rename(columns = {"gene" : "gene_ids"})
    tmp = tmp.merge(df_var, how = 'inner', on='gene_ids')

    n_infll = np.sum(tmp['ll'] == np.float64('-inf'))
    print(f'There are {n_infll} -inf log likelihood values. Filtering these out now...')
    tmp = tmp.loc[tmp['ll'] != np.float64('-inf')]

    n_unconverged = np.sum(tmp['coef_sd'] < 10E-100)
    print(f'There are {n_unconverged} logFC std. dev.s < 10E-100. Filtering these out now...')    
    tmp = tmp.loc[tmp['coef_sd'] > 10E-100]
    
    #Sort values by q-value
    tmp.sort_values(by='qval', ascending=True, inplace=True)
    
    de_full_res_overall[clust] = tmp.copy()
    
    #Filter by q-value to get significant DE genes
    tmp_filt = tmp.loc[tmp['qval'] < 0.05]
    de_signif_overall[clust] = tmp_filt.copy()

There are 0 -inf log likelihood values. Filtering these out now...
There are 117 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 2 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 30 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 54 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 168 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 30 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 15 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering thes

In [ ]:
# bulk-like data
bulk_res_overall = test_bulk_overall.summary()

bulk_res_overall = bulk_res_overall.rename(columns = {"gene" : "gene_ids"})
bulk_res_overall = bulk_res_overall.merge(df_var, how = 'inner', on='gene_ids')

n_infll = np.sum(bulk_res_overall['ll'] == np.float64('-inf'))
print(f'There are {n_infll} -inf log likelihood values. Filtering these out now...')
bulk_res_overall = bulk_res_overall.loc[bulk_res_overall['ll'] != np.float64('-inf')]

n_unconverged = np.sum(bulk_res_overall['coef_sd'] < 10E-100)
print(f'There are {n_unconverged} logFC std. dev.s < 10E-100. Filtering these out now...')    
bulk_res_overall = bulk_res_overall.loc[bulk_res_overall['coef_sd'] > 10E-100]

#Sort values by q-value
bulk_res_overall.sort_values(by='qval', ascending=True, inplace=True)
        
#Filter by q-value to get significant DE genes
bulk_res_filt_overall = bulk_res_overall.loc[bulk_res_overall['qval'] < 0.05]


### Show DEGs

In [25]:
# Number of differentially expressed genes (DEGs)
print('DEGs at FDR < 0.05:')
{clust:de_signif_overall[clust].shape[0] for clust in de_signif_overall}

print('DEGs at FDR < 0.01:')
{clust:de_signif_overall[clust].loc[(de_signif_overall[clust]['qval']<0.01)].shape[0] for clust in de_signif_overall}

DEGs at FDR < 0.05:


{'Apop.-NCSC': 209,
 'Apop.-NSC': 3033,
 'Glial-precursors': 995,
 'Immature-neurons': 2081,
 'NCSC': 774,
 'NSC1a': 9747,
 'NSC1b': 9791,
 'NSC2a': 3843,
 'NSC2b': 3639}

DEGs at FDR < 0.01:


{'Apop.-NCSC': 169,
 'Apop.-NSC': 2758,
 'Glial-precursors': 775,
 'Immature-neurons': 1664,
 'NCSC': 610,
 'NSC1a': 8964,
 'NSC1b': 9033,
 'NSC2a': 3127,
 'NSC2b': 2885}

In [ ]:
#DEGs in bulk-like data
print('DEGs at FDR < 0.05:')
print(f'Bulk: {bulk_res_filt_overall.shape[0]}')

print('DEGs at FDR < 0.01:')
degnum001 = bulk_res_filt_overall.loc[(bulk_res_filt_overall['qval']<0.01)].shape[0]
print(f'Bulk: {degnum001}')

Notes:
- there are quite a lot of DEGs at FDR < 0.05 and at FDR < 0.01
- Could use either threshold, FDR < 0.01 would be okay for higher confidence

### Save data

In [26]:
import pickle

In [27]:
#Pickle the de output
filename_pickle = save_dir+'diffxpy/'+resolution+'_diffxpy_cell_types_sf_overall.pkl'
pickle.dump(de_results_overall, open(filename_pickle, "wb"))

In [28]:
#Output individual results
for clust in de_signif_overall:
    
    filename_signif = save_dir+'diffxpy/'+resolution+'_diffxpy_'+clust.replace(' ', '_').replace(
        '/','_').replace('(','').replace(')','')+'_diff_res_overall.csv'
    filename_full = save_dir+'diffxpy/'+resolution+'_diffxpy_'+clust.replace(' ', '_').replace(
        '/','_').replace('(','').replace(')','')+'_res-table_overall.csv'
    
    de_signif_overall[clust].to_csv(filename_signif)
    de_full_res_overall[clust].to_csv(filename_full)

In [ ]:
#Pick the DE output - Bulk-like
de_output_pickle_bulk = save_dir+'diffxpy/'+resolution+'_diffxpy_bulk_sf_overall.pkl'
pickle.dump(test_bulk_overall, open(de_output_pickle_bulk, "wb"))

In [ ]:
#Output individual results - Bulk-like

filename_signif = save_dir+'diffxpy/'+resolution+'_diffxpy_bulk_like_diff_res_overall.csv'
filename_full = save_dir+'diffxpy/'+resolution+'_diffxpy_bulk_like_res-table_overall.csv'
    
bulk_res_overall_filt.iloc[:,[0,2,3,4,8]].to_csv(filename_signif)
bulk_res_overall.iloc[:,[0,2,3,4,8]].to_csv(filename_full)

In [ ]:
#del bulk_res_overall_filt
#del bulk_res_overall
#del test_bulk_overall
del de_signif_overall
del de_full_res_overall
del de_results_overall

## Differential expression testing (ctrl)

### Cell type specific DE testing

In [22]:
adata_ctrl = adata[adata.obs['condition'] == 'Control',:].copy()

In [23]:
de_results_ctrl = dict()


In [24]:
de.utils.preview_coef_names(
    sample_description=adata_ctrl.obs,
    formula="~1 + cultivation_period + donor_period", 
)

['Intercept',
 'cultivation_period[T.hp]',
 'donor_period[T.UKERi1E4-R1-003_lp]',
 'donor_period[T.UKERi1JF-R1-018_hp]',
 'donor_period[T.UKERi1JF-R1-018_lp]',
 'donor_period[T.UKERi82A-R1-002_hp]',
 'donor_period[T.UKERi82A-R1-002_lp]',
 'donor_period[T.UKERiG3G-R1-039_hp]',
 'donor_period[T.UKERiG3G-R1-039_lp]',
 'donor_period[T.UKERiO3H-R1-005_hp]',
 'donor_period[T.UKERiO3H-R1-005_lp]']

In [23]:
for clust in adata_ctrl.obs['cell_types_used'].cat.categories:
    adata_tmp = adata_ctrl[adata_ctrl.obs['cell_types_used'] == clust,:].copy()

    print(f'In cluster {clust}:')
    print(adata_tmp.obs.condition.value_counts())

    # Filter out genes to reduce multiple testing burden
    sc.pp.filter_genes(adata_tmp, min_cells=50)
    print(f'Testing {adata_tmp.n_vars} genes...')
    print("")
    
    test_tmp = de.test.wald(
    data=adata_tmp.layers['soupX_counts'],
    formula_loc="~ 1 + cultivation_period + donor_period + size_factors",
    as_numeric=['size_factors'],
    coef_to_test=['cultivation_period[T.hp]'],
    sample_description=adata_tmp.obs,
    gene_names=adata_tmp.var['gene_ids'].astype(str),
    constraints_loc={'donor_period':'cultivation_period'},
    noise_model='nb',
    dtype="float64"
    )
    
    #Store the results
    de_results_ctrl[clust] = test_tmp
    

In cluster Apop.-NCSC:
condition
Control    167
Name: count, dtype: int64
filtered out 15483 genes that are detected in less than 50 cells


Built constraints: donor_period[UKERi1E4-R1-003_lp]+donor_period[UKERi1JF-R1-018_lp]+donor_period[UKERi82A-R1-002_lp]+donor_period[UKERiG3G-R1-039_lp]+donor_period[UKERiO3H-R1-005_lp]=0, donor_period[UKERi1E4-R1-003_hp]+donor_period[UKERi82A-R1-002_hp]+donor_period[UKERiO3H-R1-005_hp]=0


Testing 1883 genes...

training location model: True
training scale model: True
iter   0: ll=1338551.702226
iter   1: ll=1314965.074088, converged: 0.00% (loc: 1.86%, scale update: False), in 0.37sec
iter   2: ll=1306903.201372, converged: 0.00% (loc: 3.93%, scale update: False), in 0.34sec
iter   3: ll=1304785.228724, converged: 0.00% (loc: 3.93%, scale update: False), in 0.33sec
iter   4: ll=1304227.070731, converged: 0.00% (loc: 3.93%, scale update: False), in 0.33sec
iter   5: ll=1304090.308958, converged: 0.00% (loc: 4.14%, scale update: False), in 0.34sec
Fitting 1883 dispersion models: (progress not available with multiprocessing)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


iter   6: ll=439429.086393, converged: 0.05% (loc: 0.05%, scale update: True), in 4.45sec
iter   7: ll=438913.749387, converged: 0.05% (loc: 4.46%, scale update: False), in 0.33sec
iter   8: ll=438822.104313, converged: 0.05% (loc: 6.64%, scale update: False), in 0.35sec
iter   9: ll=438804.069207, converged: 0.05% (loc: 11.42%, scale update: False), in 0.33sec
iter  10: ll=438799.520397, converged: 0.05% (loc: 27.19%, scale update: False), in 0.33sec
iter  11: ll=438798.378712, converged: 0.05% (loc: 36.80%, scale update: False), in 0.33sec
Fitting 1882 dispersion models: (progress not available with multiprocessing)

...

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


iter  55: ll=4926826.745041, converged: 99.91% (loc: 99.91%, scale update: True), in 2.59sec
iter  56: ll=4926822.731059, converged: 99.91% (loc: 99.98%, scale update: False), in 0.24sec
iter  57: ll=4926822.731059, converged: 99.91% (loc: 100.00%, scale update: False), in 0.19sec
Fitting 9 dispersion models: (progress not available with multiprocessing)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


iter  58: ll=4926822.731058, converged: 99.97% (loc: 99.97%, scale update: True), in 2.33sec
iter  59: ll=4926822.731058, converged: 99.97% (loc: 99.99%, scale update: False), in 0.22sec
iter  60: ll=4926822.731058, converged: 99.97% (loc: 100.00%, scale update: False), in 0.17sec
iter  61: ll=4926822.731058, converged: 99.99% (loc: 99.99%, scale update: True), in 0.21sec
iter  62: ll=4926822.731058, converged: 99.99% (loc: 99.99%, scale update: False), in 0.17sec
iter  63: ll=4926822.731058, converged: 99.99% (loc: 100.00%, scale update: False), in 0.17sec
iter  64: ll=4926822.731058, converged: 99.99% (loc: 99.99%, scale update: True), in 0.16sec
iter  65: ll=4926822.731057, converged: 99.99% (loc: 100.00%, scale update: False), in 0.18sec
iter  66: ll=4926822.731057, converged: 100.00% (loc: 100.00%, scale update: True), in 0.15sec


/usr/local/lib/python3.10/dist-packages/dask/array/core.py:2894: RuntimeWarning: divide by zero encountered in divide
  size = (limit / dtype.itemsize / largest_block) ** (1 / len(autos))


### Bulk-like DE testing across all clusters

In [25]:
# Make a copy for the testing
adata_test = adata_ctrl.copy()

print(adata_test.obs.condition.value_counts())

# Filter out genes to reduce multiple testing burden
sc.pp.filter_genes(adata_test, min_cells=50)
print(f'Testing {adata_test.n_vars} genes...')
print("")
    
test_bulk_ctrl = de.test.wald(
    data=adata_test.layers['soupX_counts'],
    formula_loc="~ 1 + cultivation_period + donor_period + size_factors",
    as_numeric=['size_factors'],
    coef_to_test=['cultivation_period[T.hp]'],
    sample_description=adata_test.obs,
    gene_names=adata_test.var['gene_ids'].astype(str),
    constraints_loc={'donor_period':'cultivation_period'},
    noise_model='nb',
    dtype="float64"
)


condition
Control    23368
Name: count, dtype: int64
filtered out 1599 genes that are detected in less than 50 cells


Built constraints: donor_period[UKERi1E4-R1-003_lp]+donor_period[UKERi1JF-R1-018_lp]+donor_period[UKERi82A-R1-002_lp]+donor_period[UKERiG3G-R1-039_lp]+donor_period[UKERiO3H-R1-005_lp]=0, donor_period[UKERi1E4-R1-003_hp]+donor_period[UKERi1JF-R1-018_hp]+donor_period[UKERi82A-R1-002_hp]+donor_period[UKERiG3G-R1-039_hp]+donor_period[UKERiO3H-R1-005_hp]=0


Testing 15767 genes...



/usr/local/lib/python3.10/dist-packages/batchglm/models/base_glm/utils.py:158: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  np.mean(x[np.where(grouping == g)[0], :], axis=0)
/usr/local/lib/python3.10/dist-packages/batchglm/models/base_glm/utils.py:171: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  expect_xsq = np.vstack([

training location model: True
training scale model: True
iter   0: ll=292249401.119247
iter   1: ll=279781133.321090, converged: 0.00% (loc: 4.75%, scale update: False), in 129.20sec
iter   2: ll=276415621.596965, converged: 0.00% (loc: 5.77%, scale update: False), in 125.10sec
iter   3: ll=275757567.158237, converged: 0.00% (loc: 5.77%, scale update: False), in 122.81sec
iter   4: ll=275627370.643273, converged: 0.00% (loc: 5.79%, scale update: False), in 120.50sec
iter   5: ll=275603485.930674, converged: 0.00% (loc: 6.33%, scale update: False), in 124.02sec
Fitting 15767 dispersion models: (progress not available with multiprocessing)

...

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


iter  42: ll=265534946.856893, converged: 99.89% (loc: 99.89%, scale update: True), in 104.11sec
caught 3 linalg singular matrix errors
iter  43: ll=265534946.856893, converged: 99.89% (loc: 99.99%, scale update: False), in 3.54sec
iter  44: ll=265534946.856892, converged: 99.89% (loc: 99.99%, scale update: False), in 1.62sec
iter  45: ll=265534946.856892, converged: 99.89% (loc: 99.99%, scale update: False), in 1.54sec
iter  46: ll=265534946.856892, converged: 99.89% (loc: 100.00%, scale update: False), in 1.65sec
Fitting 18 dispersion models: (progress not available with multiprocessing)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


iter  47: ll=265534946.856892, converged: 99.99% (loc: 99.99%, scale update: True), in 74.51sec
iter  48: ll=265534946.856892, converged: 99.99% (loc: 100.00%, scale update: False), in 1.83sec
iter  49: ll=265534946.856892, converged: 100.00% (loc: 100.00%, scale update: True), in 1.88sec


/usr/local/lib/python3.10/dist-packages/dask/array/core.py:2894: RuntimeWarning: divide by zero encountered in divide
  size = (limit / dtype.itemsize / largest_block) ** (1 / len(autos))


### Post-process data

In [26]:
# extract gene symbol - gene_ids informations for later conversion
df_var = pd.DataFrame(adata.var)
df_var = df_var[['gene_symbols', 'gene_ids']]
df_var.reset_index(inplace=True) 
df_var = df_var.drop(columns='index')

In [26]:
de_signif_ctrl = dict()
de_full_res_ctrl = dict()

for clust in de_results_ctrl:
    tmp = de_results_ctrl[clust].summary()
    
    # add gene symbols
    tmp = tmp.rename(columns = {"gene" : "gene_ids"})
    tmp = tmp.merge(df_var, how = 'inner', on='gene_ids')

    n_infll = np.sum(tmp['ll'] == np.float64('-inf'))
    print(f'There are {n_infll} -inf log likelihood values. Filtering these out now...')
    tmp = tmp.loc[tmp['ll'] != np.float64('-inf')]

    n_unconverged = np.sum(tmp['coef_sd'] < 10E-100)
    print(f'There are {n_unconverged} logFC std. dev.s < 10E-100. Filtering these out now...')    
    tmp = tmp.loc[tmp['coef_sd'] > 10E-100]
    
    #Sort values by q-value
    tmp.sort_values(by='qval', ascending=True, inplace=True)
    
    de_full_res_ctrl[clust] = tmp.copy()
    
    #Filter by q-value to get significant DE genes
    tmp_filt = tmp.loc[tmp['qval'] < 0.05]
    de_signif_ctrl[clust] = tmp_filt.copy()

There are 0 -inf log likelihood values. Filtering these out now...
There are 7 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 1 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 3 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 0 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 106 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 10 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 2 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out

In [27]:
# bulk-like data
bulk_res_ctrl = test_bulk_ctrl.summary()

bulk_res_ctrl = bulk_res_ctrl.rename(columns = {"gene" : "gene_ids"})
bulk_res_ctrl = bulk_res_ctrl.merge(df_var, how = 'inner', on='gene_ids')

n_infll = np.sum(bulk_res_ctrl['ll'] == np.float64('-inf'))
print(f'There are {n_infll} -inf log likelihood values. Filtering these out now...')
bulk_res_ctrl = bulk_res_ctrl.loc[bulk_res_ctrl['ll'] != np.float64('-inf')]

n_unconverged = np.sum(bulk_res_ctrl['coef_sd'] < 10E-100)
print(f'There are {n_unconverged} logFC std. dev.s < 10E-100. Filtering these out now...')    
bulk_res_ctrl = bulk_res_ctrl.loc[bulk_res_ctrl['coef_sd'] > 10E-100]

#Sort values by q-value
bulk_res_ctrl.sort_values(by='qval', ascending=True, inplace=True)
        
#Filter by q-value to get significant DE genes
bulk_res_filt_ctrl = bulk_res_ctrl.loc[bulk_res_ctrl['qval'] < 0.05]


There are 0 -inf log likelihood values. Filtering these out now...
There are 4 logFC std. dev.s < 10E-100. Filtering these out now...


### Show DEGs

In [28]:
# Number of differentially expressed genes (DEGs)
print('DEGs at FDR < 0.05:')
{clust:de_signif_ctrl[clust].shape[0] for clust in de_signif_ctrl}

print('DEGs at FDR < 0.01:')
{clust:de_signif_ctrl[clust].loc[(de_signif_ctrl[clust]['qval']<0.01)].shape[0] for clust in de_signif_ctrl}

DEGs at FDR < 0.05:


{'Apop.-NCSC': 71,
 'Apop.-NSC': 3483,
 'Glial-precursors': 149,
 'Immature-neurons': 2666,
 'NCSC': 300,
 'NSC1a': 8231,
 'NSC1b': 8500,
 'NSC2a': 1891,
 'NSC2b': 1690}

DEGs at FDR < 0.01:


{'Apop.-NCSC': 49,
 'Apop.-NSC': 2851,
 'Glial-precursors': 85,
 'Immature-neurons': 1913,
 'NCSC': 181,
 'NSC1a': 7223,
 'NSC1b': 7488,
 'NSC2a': 1310,
 'NSC2b': 1101}

In [29]:
#DEGs in bulk-like data
print('DEGs at FDR < 0.05:')
print(f'Bulk: {bulk_res_filt_ctrl.shape[0]}')

print('DEGs at FDR < 0.01:')
degnum001 = bulk_res_filt_ctrl.loc[(bulk_res_filt_ctrl['qval']<0.01)].shape[0]
print(f'Bulk: {degnum001}')

DEGs at FDR < 0.05:
Bulk: 12075
DEGs at FDR < 0.01:
Bulk: 11325


Notes:
- there are quite a lot of DEGs at FDR < 0.05 and at FDR < 0.01
- Could use either threshold, FDR < 0.01 would be okay for higher confidence

### Save data

In [30]:
import pickle

In [31]:
#Pickle the de output
filename_pickle = save_dir+'diffxpy/'+resolution+'_diffxpy_cell_types_sf_ctrl.pkl'
pickle.dump(de_results_ctrl, open(filename_pickle, "wb"))

In [32]:
#Output individual results
for clust in de_signif_ctrl:
    
    filename_signif = save_dir+'diffxpy/'+resolution+'_diffxpy_'+clust.replace(' ', '_').replace(
        '/','_').replace('(','').replace(')','')+'_diff_res_ctrl.csv'
    filename_full = save_dir+'diffxpy/'+resolution+'_diffxpy_'+clust.replace(' ', '_').replace(
        '/','_').replace('(','').replace(')','')+'_res-table_ctrl.csv'
    
    de_signif_ctrl[clust].to_csv(filename_signif)
    de_full_res_ctrl[clust].to_csv(filename_full)

In [33]:
#Pick the DE output - Bulk-like
de_output_pickle_bulk = save_dir+'diffxpy/'+resolution+'_diffxpy_bulk_sf_ctrl.pkl'
pickle.dump(test_bulk_ctrl, open(de_output_pickle_bulk, "wb"))

In [28]:
#Output individual results - Bulk-like

filename_signif = save_dir+'diffxpy/'+resolution+'_diffxpy_bulk_like_diff_res_ctrl.csv'
filename_full = save_dir+'diffxpy/'+resolution+'_diffxpy_bulk_like_res-table_ctrl.csv'
    
bulk_res_filt_ctrl.to_csv(filename_signif)
bulk_res_ctrl.to_csv(filename_full)

## Differential expression testing (PD)

### Cell type specific DE testing

In [20]:
adata_pd = adata[adata.obs['condition'] == 'Parkinson',:].copy()

In [21]:
de_results_pd = dict()


In [22]:
de.utils.preview_coef_names(
    sample_description=adata_pd.obs,
    formula="~1 + cultivation_period + donor_period", 
)

['Intercept',
 'cultivation_period[T.hp]',
 'donor_period[T.UKERi88H-R1-002_lp]',
 'donor_period[T.UKERiAY6-R1-003_hp]',
 'donor_period[T.UKERiAY6-R1-003_lp]',
 'donor_period[T.UKERiC99-R1-007_hp]',
 'donor_period[T.UKERiC99-R1-007_lp]',
 'donor_period[T.UKERiJ2C-R1-015_hp]',
 'donor_period[T.UKERiJ2C-R1-015_lp]',
 'donor_period[T.UKERiM89-R1-005_hp]',
 'donor_period[T.UKERiM89-R1-005_lp]',
 'donor_period[T.UKERiPX7-R1-001_hp]',
 'donor_period[T.UKERiPX7-R1-001_lp]',
 'donor_period[T.UKERiR66-R1-007_hp]',
 'donor_period[T.UKERiR66-R1-007_lp]']

In [22]:
for clust in adata_pd.obs['cell_types_used'].cat.categories:
    adata_tmp = adata_pd[adata_pd.obs['cell_types_used'] == clust,:].copy()

    print(f'In cluster {clust}:')
    print(adata_tmp.obs.condition.value_counts())

    # Filter out genes to reduce multiple testing burden
    sc.pp.filter_genes(adata_tmp, min_cells=50)
    print(f'Testing {adata_tmp.n_vars} genes...')
    print("")
    
    test_tmp = de.test.wald(
    data=adata_tmp.layers['soupX_counts'],
    formula_loc="~ 1 + cultivation_period + donor_period + size_factors",
    as_numeric=['size_factors'],
    coef_to_test=['cultivation_period[T.hp]'],
    sample_description=adata_tmp.obs,
    gene_names=adata_tmp.var['gene_ids'].astype(str),
    constraints_loc={'donor_period':'cultivation_period'},
    noise_model='nb',
    dtype="float64"
    )
    
    #Store the results
    de_results_pd[clust] = test_tmp
    

In cluster Apop.-NCSC:
condition
Parkinson    159
Name: count, dtype: int64
filtered out 15440 genes that are detected in less than 50 cells


Built constraints: donor_period[UKERi88H-R1-002_lp]+donor_period[UKERiAY6-R1-003_lp]+donor_period[UKERiC99-R1-007_lp]+donor_period[UKERiJ2C-R1-015_lp]+donor_period[UKERiM89-R1-005_lp]+donor_period[UKERiPX7-R1-001_lp]+donor_period[UKERiR66-R1-007_lp]=0, donor_period[UKERi88H-R1-002_hp]+donor_period[UKERiAY6-R1-003_hp]+donor_period[UKERiC99-R1-007_hp]+donor_period[UKERiJ2C-R1-015_hp]+donor_period[UKERiPX7-R1-001_hp]+donor_period[UKERiR66-R1-007_hp]=0


Testing 1926 genes...

training location model: True
training scale model: True
iter   0: ll=2685697.930719
iter   1: ll=2657741.079191, converged: 0.00% (loc: 0.52%, scale update: False), in 0.72sec
iter   2: ll=2651055.810864, converged: 0.00% (loc: 5.45%, scale update: False), in 0.62sec
iter   3: ll=2649985.348373, converged: 0.00% (loc: 5.45%, scale update: False), in 0.65sec
iter   4: ll=2649759.621047, converged: 0.00% (loc: 5.45%, scale update: False), in 0.59sec
iter   5: ll=2649696.073684, converged: 0.00% (loc: 8.41%, scale update: False), in 0.62sec
Fitting 1926 dispersion models: (progress not available with multiprocessing)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


iter   6: ll=424303.758551, converged: 0.00% (loc: 0.00%, scale update: True), in 5.08sec
iter   7: ll=423780.222091, converged: 0.00% (loc: 4.78%, scale update: False), in 0.67sec
iter   8: ll=423725.373193, converged: 0.00% (loc: 9.71%, scale update: False), in 0.61sec
iter   9: ll=423715.464775, converged: 0.00% (loc: 21.13%, scale update: False), in 0.75sec
iter  10: ll=423711.388167, converged: 0.00% (loc: 43.93%, scale update: False), in 0.58sec
iter  11: ll=423708.806336, converged: 0.00% (loc: 61.73%, scale update: False), in 0.36sec
Fitting 1926 dispersion models: (progress not available with multiprocessing)

...

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


iter  41: ll=21589207.073258, converged: 99.90% (loc: 99.90%, scale update: True), in 8.87sec
caught 3 linalg singular matrix errors
iter  42: ll=21589206.544281, converged: 99.90% (loc: 99.98%, scale update: False), in 0.63sec
iter  43: ll=21589206.544281, converged: 99.90% (loc: 100.00%, scale update: False), in 0.41sec
Fitting 13 dispersion models: (progress not available with multiprocessing)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


iter  44: ll=21589027.892233, converged: 99.98% (loc: 99.98%, scale update: True), in 6.34sec
iter  45: ll=21589027.892233, converged: 99.98% (loc: 100.00%, scale update: False), in 0.38sec
iter  46: ll=21589027.892233, converged: 100.00% (loc: 100.00%, scale update: True), in 0.49sec


/usr/local/lib/python3.10/dist-packages/dask/array/core.py:2894: RuntimeWarning: divide by zero encountered in divide
  size = (limit / dtype.itemsize / largest_block) ** (1 / len(autos))


### Bulk-like DE testing across all clusters

In [23]:
# Make a copy for the testing
adata_test = adata_pd.copy()

print(adata_test.obs.condition.value_counts())

# Filter out genes to reduce multiple testing burden
sc.pp.filter_genes(adata_test, min_cells=50)
print(f'Testing {adata_test.n_vars} genes...')
print("")
    
test_bulk_pd = de.test.wald(
    data=adata_test.layers['soupX_counts'],
    formula_loc="~ 1 + cultivation_period + donor_period + size_factors",
    as_numeric=['size_factors'],
    coef_to_test=['cultivation_period[T.hp]'],
    sample_description=adata_test.obs,
    gene_names=adata_test.var['gene_ids'].astype(str),
    constraints_loc={'donor_period':'cultivation_period'},
    noise_model='nb',
    dtype="float64"
)


condition
Parkinson    38032
Name: count, dtype: int64
filtered out 1083 genes that are detected in less than 50 cells


Built constraints: donor_period[UKERi88H-R1-002_lp]+donor_period[UKERiAY6-R1-003_lp]+donor_period[UKERiC99-R1-007_lp]+donor_period[UKERiJ2C-R1-015_lp]+donor_period[UKERiM89-R1-005_lp]+donor_period[UKERiPX7-R1-001_lp]+donor_period[UKERiR66-R1-007_lp]=0, donor_period[UKERi88H-R1-002_hp]+donor_period[UKERiAY6-R1-003_hp]+donor_period[UKERiC99-R1-007_hp]+donor_period[UKERiJ2C-R1-015_hp]+donor_period[UKERiM89-R1-005_hp]+donor_period[UKERiPX7-R1-001_hp]+donor_period[UKERiR66-R1-007_hp]=0


Testing 16283 genes...



/usr/local/lib/python3.10/dist-packages/batchglm/models/base_glm/utils.py:158: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  np.mean(x[np.where(grouping == g)[0], :], axis=0)
/usr/local/lib/python3.10/dist-packages/batchglm/models/base_glm/utils.py:171: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  expect_xsq = np.vstack([

training location model: True
training scale model: True
iter   0: ll=485520116.223383
iter   1: ll=465617480.641590, converged: 0.00% (loc: 2.73%, scale update: False), in 270.75sec
iter   2: ll=461746466.899396, converged: 0.00% (loc: 4.00%, scale update: False), in 226.73sec
iter   3: ll=461212873.705327, converged: 0.00% (loc: 4.00%, scale update: False), in 236.13sec
iter   4: ll=461134087.392649, converged: 0.00% (loc: 4.02%, scale update: False), in 225.91sec
iter   5: ll=461123032.191936, converged: 0.00% (loc: 5.59%, scale update: False), in 227.86sec
Fitting 16283 dispersion models: (progress not available with multiprocessing)

...

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


iter  46: ll=448704198.269964, converged: 99.98% (loc: 99.98%, scale update: True), in 124.97sec
caught 1 linalg singular matrix errors
iter  47: ll=448704198.269787, converged: 99.98% (loc: 99.99%, scale update: False), in 3.17sec
iter  48: ll=448704198.269786, converged: 99.98% (loc: 99.99%, scale update: False), in 2.78sec
iter  49: ll=448704198.269786, converged: 99.98% (loc: 100.00%, scale update: False), in 3.15sec
Fitting 4 dispersion models: (progress not available with multiprocessing)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


iter  50: ll=448704198.269689, converged: 99.99% (loc: 99.99%, scale update: True), in 70.54sec
iter  51: ll=448704198.269636, converged: 99.99% (loc: 99.99%, scale update: False), in 2.80sec
iter  52: ll=448704198.269636, converged: 99.99% (loc: 99.99%, scale update: False), in 2.77sec
iter  53: ll=448704198.269636, converged: 99.99% (loc: 100.00%, scale update: False), in 2.80sec
iter  54: ll=448704025.654632, converged: 99.99% (loc: 99.99%, scale update: True), in 2.73sec
iter  55: ll=448704025.654632, converged: 99.99% (loc: 100.00%, scale update: False), in 2.77sec
iter  56: ll=448703420.330359, converged: 99.99% (loc: 99.99%, scale update: True), in 2.82sec
iter  57: ll=448703420.330359, converged: 99.99% (loc: 100.00%, scale update: False), in 2.80sec
iter  58: ll=448703420.330359, converged: 100.00% (loc: 100.00%, scale update: True), in 2.72sec


/usr/local/lib/python3.10/dist-packages/dask/array/core.py:2894: RuntimeWarning: divide by zero encountered in divide
  size = (limit / dtype.itemsize / largest_block) ** (1 / len(autos))


### Post-process data

In [24]:
# extract gene symbol - gene_ids informations for later conversion
df_var = pd.DataFrame(adata.var)
df_var = df_var[['gene_symbols', 'gene_ids']]
df_var.reset_index(inplace=True) 
df_var = df_var.drop(columns='index')

In [27]:
de_signif_pd = dict()
de_full_res_pd = dict()

for clust in de_results_pd:
    tmp = de_results_pd[clust].summary()
    
    # add gene symbols
    tmp = tmp.rename(columns = {"gene" : "gene_ids"})
    tmp = tmp.merge(df_var, how = 'inner', on='gene_ids')

    n_infll = np.sum(tmp['ll'] == np.float64('-inf'))
    print(f'There are {n_infll} -inf log likelihood values. Filtering these out now...')
    tmp = tmp.loc[tmp['ll'] != np.float64('-inf')]

    n_unconverged = np.sum(tmp['coef_sd'] < 10E-100)
    print(f'There are {n_unconverged} logFC std. dev.s < 10E-100. Filtering these out now...')    
    tmp = tmp.loc[tmp['coef_sd'] > 10E-100]
    
    #Sort values by q-value
    tmp.sort_values(by='qval', ascending=True, inplace=True)
    
    de_full_res_pd[clust] = tmp.copy()
    
    #Filter by q-value to get significant DE genes
    tmp_filt = tmp.loc[tmp['qval'] < 0.05]
    de_signif_pd[clust] = tmp_filt.copy()

There are 0 -inf log likelihood values. Filtering these out now...
There are 1 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 2 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 5 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 28 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 9 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 8 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 4 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out n

In [25]:
# bulk-like data
bulk_res_pd = test_bulk_pd.summary()

bulk_res_pd = bulk_res_pd.rename(columns = {"gene" : "gene_ids"})
bulk_res_pd = bulk_res_pd.merge(df_var, how = 'inner', on='gene_ids')

n_infll = np.sum(bulk_res_pd['ll'] == np.float64('-inf'))
print(f'There are {n_infll} -inf log likelihood values. Filtering these out now...')
bulk_res_pd = bulk_res_pd.loc[bulk_res_pd['ll'] != np.float64('-inf')]

n_unconverged = np.sum(bulk_res_pd['coef_sd'] < 10E-100)
print(f'There are {n_unconverged} logFC std. dev.s < 10E-100. Filtering these out now...')    
bulk_res_pd = bulk_res_pd.loc[bulk_res_pd['coef_sd'] > 10E-100]

#Sort values by q-value
bulk_res_pd.sort_values(by='qval', ascending=True, inplace=True)
        
#Filter by q-value to get significant DE genes
bulk_res_filt_pd = bulk_res_pd.loc[bulk_res_pd['qval'] < 0.05]


There are 0 -inf log likelihood values. Filtering these out now...
There are 3 logFC std. dev.s < 10E-100. Filtering these out now...


### Show DEGs

In [29]:
# Number of differentially expressed genes (DEGs)
print('DEGs at FDR < 0.05:')
{clust:de_signif_pd[clust].shape[0] for clust in de_signif_pd}

print('DEGs at FDR < 0.01:')
{clust:de_signif_pd[clust].loc[(de_signif_pd[clust]['qval']<0.01)].shape[0] for clust in de_signif_pd}

DEGs at FDR < 0.05:


{'Apop.-NCSC': 286,
 'Apop.-NSC': 2541,
 'Glial-precursors': 2275,
 'Immature-neurons': 1318,
 'NCSC': 6442,
 'NSC1a': 10005,
 'NSC1b': 9711,
 'NSC2a': 7169,
 'NSC2b': 4118}

DEGs at FDR < 0.01:


{'Apop.-NCSC': 189,
 'Apop.-NSC': 2030,
 'Glial-precursors': 1669,
 'Immature-neurons': 1003,
 'NCSC': 5352,
 'NSC1a': 9072,
 'NSC1b': 8767,
 'NSC2a': 6101,
 'NSC2b': 3138}

In [30]:
#DEGs in bulk-like data
print('DEGs at FDR < 0.05:')
print(f'Bulk: {bulk_res_filt_pd.shape[0]}')

print('DEGs at FDR < 0.01:')
degnum001 = bulk_res_filt_pd.loc[(bulk_res_filt_pd['qval']<0.01)].shape[0]
print(f'Bulk: {degnum001}')

DEGs at FDR < 0.05:
Bulk: 12562
DEGs at FDR < 0.01:
Bulk: 11881


Notes:
- there are quite a lot of DEGs at FDR < 0.05 and at FDR < 0.01
- Could use either threshold, FDR < 0.01 would be okay for higher confidence

### Save data

In [31]:
import pickle

In [32]:
#Pickle the de output
filename_pickle = save_dir+'diffxpy/'+resolution+'_diffxpy_cell_types_sf_pd.pkl'
pickle.dump(de_results_pd, open(filename_pickle, "wb"))

In [33]:
#Output individual results
for clust in de_signif_pd:
    
    filename_signif = save_dir+'diffxpy/'+resolution+'_diffxpy_'+clust.replace(' ', '_').replace(
        '/','_').replace('(','').replace(')','')+'_diff_res_pd.csv'
    filename_full = save_dir+'diffxpy/'+resolution+'_diffxpy_'+clust.replace(' ', '_').replace(
        '/','_').replace('(','').replace(')','')+'_res-table_pd.csv'
    
    de_signif_pd[clust].to_csv(filename_signif)
    de_full_res_pd[clust].to_csv(filename_full)

In [34]:
#Pick the DE output - Bulk-like
de_output_pickle_bulk = save_dir+'diffxpy/'+resolution+'_diffxpy_bulk_sf_pd.pkl'
pickle.dump(test_bulk_pd, open(de_output_pickle_bulk, "wb"))

In [26]:
#Output individual results - Bulk-like

filename_signif = save_dir+'diffxpy/'+resolution+'_diffxpy_bulk_like_diff_res_pd.csv'
filename_full = save_dir+'diffxpy/'+resolution+'_diffxpy_bulk_like_res-table_pd.csv'
    
bulk_res_filt_pd.to_csv(filename_signif)
bulk_res_pd.to_csv(filename_full)

R[write to console]: Warning message:

R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  library ‘/usr/lib/R/site-library’ contains no packages



In [41]:
!pip install dask==2023.11.0 sparse==0.15.1

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: sparse
    Found existing installation: sparse 0.9.1
    Uninstalling sparse-0.9.1:
      Successfully uninstalled sparse-0.9.1
  Attempting uninstall: dask
    Found existing installation: dask 2021.4.0
    Uninstalling dask-2021.4.0:
      Successfully uninstalled dask-2021.4.0
